In [ ]:

# -*- coding: utf-8 -*-
"""
Google Colab için Tam Entegre Veri Toplama Sistemi
"""

# 1. GEREKLİ PAKETLERİN KURULUMU
!pip install arxiv semanticscholar --quiet  # Sessiz kurulum için --quiet parametresi

# 2. GOOGLE DRIVE ENTEGRASYONU
from google.colab import drive
drive.mount('/content/drive')

# 3. KÜTÜPHANELERİN YÜKLENMESİ
import arxiv
import json
import pandas as pd
from semanticscholar import SemanticScholar
from tqdm import tqdm  # İlerleme çubuğu için
import time  # API istekleri arası bekleme süresi

# 4. YAPILANDIRMA PARAMETRELERİ
CONFIG = {
    "search_terms": [
        "diamond based quantum computing",
        "NV centers in diamond",
        "quantum sensors with diamonds",
        "spin qubits diamond",
        "diamonds in quantum optics"
    ],
    "max_results_per_term": 150,  # Her terim için maks sonuç
    "output_formats": ["json", "csv"],  # Çıktı formatları
    "output_path": "/content/drive/MyDrive/quantum_diamond_dataset",
    "api_delay": 1  # API istekleri arası bekleme süresi (saniye)
}

# 5. VERİ TOPLAMA FONKSİYONLARI

def get_arxiv_data():
    """arXiv API'den veri çeker"""
    collected = []

    for term in tqdm(CONFIG["search_terms"], desc="arXiv Veri Toplama"):
        try:
            search = arxiv.Search(
                query=term,
                max_results=CONFIG["max_results_per_term"],
                sort_by=arxiv.SortCriterion.SubmittedDate
            )

            for result in search.results():
                collected.append({
                    "id": result.get_short_id(),
                    "title": result.title,
                    "abstract": result.summary,
                    "authors": [a.name for a in result.authors],
                    "published": result.published.isoformat() if result.published else None,
                    "doi": result.doi,
                    "pdf_url": result.pdf_url,
                    "source": "arXiv"
                })

        except Exception as e:
            print(f"\narXiv hatası ({term}): {str(e)}")
            continue

    return collected

def get_semantic_scholar_data():
    """Semantic Scholar API'den veri çeker"""
    collected = []
    scholar = SemanticScholar()

    for term in tqdm(CONFIG["search_terms"], desc="Semantic Scholar Veri Toplama"):
        try:
            results = scholar.search_paper(
                term,
                limit=CONFIG["max_results_per_term"],
                fields=["title", "abstract", "authors", "publicationDate", "openAccessPdf", "externalIds"]
            )

            time.sleep(CONFIG["api_delay"])  # API rate limit koruması

            for paper in results.data:
                # Yazar isimlerini işleme
                authors = [a['author']['name'] for a in paper['authors']] if paper.get('authors') else []

                # PDF URL kontrolü
                pdf_url = paper['openAccessPdf']['url'] if paper.get('openAccessPdf') else None

                # DOI bilgisi
                doi = paper['externalIds'].get('DOI') if paper.get('externalIds') else None

                collected.append({
                    "id": paper['paperId'],
                    "title": paper.get('title', 'No Title'),
                    "abstract": paper.get('abstract', 'No Abstract'),
                    "authors": authors,
                    "published": paper.get('publicationDate'),
                    "doi": doi,
                    "pdf_url": pdf_url,
                    "source": "Semantic Scholar"
      22 A3          })

        except Exception as e:
            print(f"\nSemantic Scholar hatası ({term}): {str(e)}")
            continue

    return collected

# 6. VERİ TOPLAMA İŞLEMLERİ
print("\n" + "="*50)
print("VERİ TOPLAMA SÜRECİ BAŞLATILIYOR")
print("="*50 + "\n")

# arXiv verilerini topla
arxiv_data = get_arxiv_data()
print(f"\narXiv'den toplanan makale sayısı: {len(arxiv_data)}")

# Semantic Scholar verilerini topla
semantic_data = get_semantic_scholar_data()
print(f"Semantic Scholar'dan toplanan makale sayısı: {len(semantic_data)}")

# Tüm verileri birleştir
combined_data = arxiv_data + semantic_data
print(f"\nToplam toplanan makale sayısı: {len(combined_data)}")

# 7. VERİ KALİTE KONTROLLERİ
print("\nVERİ KALİTE KONTROLLERİ:")
print(f"- Benzersiz makale sayısı: {len({p['id'] for p in combined_data})}")
print(f"- Özeti olmayan makaleler: {sum(1 for p in combined_data if len(p['abstract'])<100)}")
print(f"- PDF bağlantısı olanlar: {sum(1 for p in combined_data if p['pdf_url'])}")

# 8. VERİLERİ KAYDETME
print("\nVERİ KAYIT İŞLEMLERİ:")

# JSON kaydetme
if "json" in CONFIG["output_formats"]:
    json_path = f"{CONFIG['output_path']}.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(combined_data, f, ensure_ascii=False, indent=2)
    print(f"JSON dosyası kaydedildi: {json_path}")

# CSV kaydetme
if "csv" in CONFIG["output_formats"]:
    csv_path = f"{CONFIG['output_path']}.csv"
    df = pd.DataFrame(combined_data)

    # Yazar listelerini string'e çevir
    df['authors'] = df['authors'].apply(lambda x: '; '.join(x) if x else '')

    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"CSV dosyası kaydedildi: {csv_path}")

# 9. SONUÇ RAPORU
print("\n" + "="*50)
print("VERİ TOPLAMA SÜRECİ TAMAMLANDI")
print("="*50)
print(f"Toplam Kayıt Sayısı: {len(combined_data)}")
print(f"Kayıtlar Google Drive'da şu konuma kaydedildi: {CONFIG['output_path']}[.json/.csv]")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00
Mounted at /content/drive

VERİ TOPLAMA SÜRECİ BAŞLATILIYOR



arXiv Veri Toplama:   0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-1-d75d993dabee>:50: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
arXiv Veri Toplama:  40%|████      | 2/5 [00:28<00:46, 15.61s/it]


arXiv hatası (NV centers in diamond): Page of results was unexpectedly empty (https://export.arxiv.org/api/query?search_query=NV+centers+in+diamond&id_list=&sortBy=submittedDate&sortOrder=descending&start=100&max_results=100)


arXiv Veri Toplama: 100%|██████████| 5/5 [00:50<00:00, 10.14s/it]



arXiv'den toplanan makale sayısı: 700


Semantic Scholar Veri Toplama: 100%|██████████| 5/5 [00:00<00:00, 4134.76it/s]



Semantic Scholar hatası (diamond based quantum computing): The limit parameter must be between 1 and 100 inclusive.

Semantic Scholar hatası (NV centers in diamond): The limit parameter must be between 1 and 100 inclusive.

Semantic Scholar hatası (quantum sensors with diamonds): The limit parameter must be between 1 and 100 inclusive.

Semantic Scholar hatası (spin qubits diamond): The limit parameter must be between 1 and 100 inclusive.

Semantic Scholar hatası (diamonds in quantum optics): The limit parameter must be between 1 and 100 inclusive.
Semantic Scholar'dan toplanan makale sayısı: 0

Toplam toplanan makale sayısı: 700

VERİ KALİTE KONTROLLERİ:
- Benzersiz makale sayısı: 344
- Özeti olmayan makaleler: 0
- PDF bağlantısı olanlar: 700

VERİ KAYIT İŞLEMLERİ:
JSON dosyası kaydedildi: /content/drive/MyDrive/quantum_diamond_dataset.json
CSV dosyası kaydedildi: /content/drive/MyDrive/quantum_diamond_dataset.csv

VERİ TOPLAMA SÜRECİ TAMAMLANDI
Toplam Kayıt Sayısı: 700
Kayıtlar Goog